<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To check GPU
!nvidia-smi

Thu Dec 12 02:56:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-z4izym_7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-z4izym_7
  Resolved https://github.com/huggingface/transformers.git to commit a9ccdfd8e3fbaf19ad6f174a99e8d39e706950b3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10188114 sha256=ed7f0b9e373821ddf062738a1f9abc970636d2bf0300f17245b7afeb7cb80c0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-1z1qtzkd/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      S

In [3]:
!pip install --upgrade torch datasets accelerate peft bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Fo

In [4]:
import transformers
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Gated model which requires huggingface login
from huggingface_hub import login
from google.colab import userdata
access_token_read = userdata.get('hf_token') # Get access token from HuggingFace account
login(token = access_token_read)

In [8]:
# Define the model name (Mistral 7B Instruct model)
model_name = 'mistralai/Mistral-7B-Instruct-v0.1'

# Load the model configuration from the pretrained model
model_config = transformers.AutoConfig.from_pretrained(model_name)

# Initialize the tokenizer with remote code trusted (for custom tokenizer implementations)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Set the pad token to the end-of-sequence (EOS) token for better padding compatibility
tokenizer.pad_token = tokenizer.eos_token

# Specify that padding should be added on the right side (common in most NLP tasks)
tokenizer.padding_side = "right"

# Flag to indicate whether 4-bit quantization should be used for the model
use_4bit = True

# Define the computation data type for 4-bit quantization (float16 for reduced memory usage)
bnb_4bit_compute_dtype = "float16"

# Specify the quantization type; "nf4" stands for Normal Float 4 (improves precision for low-bit quantization)
bnb_4bit_quant_type = "nf4"

# Flag to control nested quantization (set to False to avoid additional quantization layers)
use_nested_quant = False

# Get the appropriate PyTorch data type (e.g., torch.float16) for computation
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Configure the BitsAndBytes (bnb) 4-bit quantization settings
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,                    # Enable 4-bit quantization
    bnb_4bit_quant_type=bnb_4bit_quant_type,  # Specify the quantization type (NF4)
    bnb_4bit_compute_dtype=compute_dtype,     # Set the computation data type (float16)
    bnb_4bit_use_double_quant=use_nested_quant # Use or disable double quantization (nested quantization)
)

# The configured tokenizer and quantization settings can now be used to load the model with optimized performance.

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=2048,
    output_scores=True
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
text_generation_pipeline("Explain to me the working of LLM")

[{'generated_text': 'Explain to me the working of LLM.\n\nI am a student and I want to know how LLM works. Can you explain it?\n\nLLM stands for Language Model. It is a type of artificial intelligence that can understand and generate human language. LLMs are trained on large amounts of text data, which allows them to learn patterns and structures in language.\n\nWhen an LLM is presented with a new piece of text, it uses its knowledge of language to predict what the next word or phrase is likely to be. This process is called "language modeling." The LLM\'s predictions are based on the probability of each possible word or phrase given the context of the previous words or phrases.\n\nOne way LLMs are used is in natural language processing (NLP) tasks such as speech recognition, machine translation, and sentiment analysis. For example, an LLM could be trained to recognize spoken English and transcribe it into written text.\n\nAnother way LLMs are used is in generating text. For example, an

In [12]:
text_generation_pipeline("What's the basics of rocket sciecne?")

[{'generated_text': "What's the basics of rocket sciecne?\n\nI'm a 14 year old who wants to learn about rocket science. I want to know what is the basics of rocket science and how do they work. Can you help me out?\n\nA: Sure, I can give you an overview of the basics of rocket science. A rocket works by expelling hot gas in one direction at high speed, which propels the rocket forward. The hot gas is produced by burning fuel, which is usually stored in tanks within the rocket.\n\nThe main components of a rocket include the fuel tanks, engines, guidance system, and payload. The engines use combustion or nuclear reactions to produce thrust, which propels the rocket forward. The guidance system controls the direction and trajectory of the rocket. The payload is the object that the rocket is designed to carry into space.\n\nThere are several different types of rockets, including solid-fueled, liquid-fueled, and hybrid rockets. Solid-fueled rockets use pre-packed fuel pellets that burn unif

In [13]:
text_generation_pipeline("Who all are part of deep state?")

[{'generated_text': 'Who all are part of deep state?\nDeep state refers to a group of people who hold significant power and influence in government, often behind the scenes. It is not a formal organization or structure, but rather a loose network of individuals and groups who share similar interests and goals. Some of the people who have been accused of being part of the deep state include politicians, intelligence agencies, military leaders, and corporate executives. However, it is important to note that these accusations are often unfounded and lack concrete evidence.'}]